In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sc

$Y = median\_house\_value \Rightarrow$ Verkliga värden, målvariabel

$X = designmatris \Rightarrow$ Alla regressorer, shape

$\hat{Y} = \hat{\beta}_0 + \hat{\beta}_1X_1 + \hat{\beta}_2X_2 + \hat{\beta}_3X_3 ... + \hat{\beta}_dX_d \Rightarrow$ Modellens prediktion för varje observation

$e_i = Y_i - \hat{Y}_i \Rightarrow$ Residual för observation $i$

$SSE = \sum_{i=1}^{n} e_i^2 \Rightarrow$ Sum of Squared Errors - totala felet för modellen

$S_{yy} = \sum(Y_i - \bar{Y})^2 = \sum{Y_{i}}^2 - \frac{(\sum{Y_i})^2}{n} \Rightarrow$ Totala variationen i Y

$SSR = S_{yy} - SSE \Rightarrow$ Regression Sum of Squares - variation som modellen förklarar

$R^2 = \frac{SSR}{S_{yy}} \Rightarrow$ Bestämningskoefficient - hur mycket av Y som modellen förklarar

$\hat{\sigma}^2 = \frac{SSE}{n-d-1} \Rightarrow$ Estimering av residualvarians

$RMSE = \sqrt{\frac{SSE}{n-d-1}} \Rightarrow$ Root Mean Squared Error - spridning i samma enhet som Y (använder $\sqrt{\frac{SSE}{n-d-1}}$ istället för $\sqrt{\frac{SSE}{n}}$ då den senare används i större utsträckning inom ML och uppgiften skulle vara en statistisk analys)

$\hat{\beta} = (X^TX)^{-1}X^TY \Rightarrow$ OLS-koefficienter - beräkning av alla $\beta$ från datan

$C = \hat{\sigma}^2(X^TX)^{-1} \Rightarrow$ Varians och kovarians för $\hat{\beta}$, används i t-test och konfidensintervall

$t_i = \frac{\hat{\beta}_i}{\hat{\sigma}\sqrt{c_{ii}}} \Rightarrow$ t-statistik - Testar om en enskild koefficient skiljer sig signifikant från 0

$F = \frac{SSR/d}{\hat{\sigma}^2} \Rightarrow$ Testar om alla regressorer tillsammans är signifikanta

$\hat{\beta}_i \pm t_{\alpha/2, n-d-1}\hat{\sigma}\sqrt{c_{ii}} \Rightarrow$ Intervallet för $\beta$ med viss säkerhetsnivå

$r = \frac{Cov(X_a,X_b)}{\sqrt{Var(X_a)Var(X_b)}} \Rightarrow$ Pearson $r$ - Mäta linjärt beroende mellan två regressorer

$d$ = antal regressorer  
$n$ = antal observationer  
$c_{ii}$ = diagonalelement i varians-kovariansmatrisen $C$  
Alla formler bygger direkt på residualerna $e_i$ och $X$-matrisen

Datasetet läses in som en dataframe och aggregeras till en observation per geografisk punkt (longitude, latitude), då flera koordinater förekommer mer än en gång. Additiva variabler summeras medan nivåvariabler ersätts av det nya medelvärdet. Den kategoriska variabeln ocean_proximity var konstant inom varje grupp vilket även verifierades och möjliggjorde oförändrad inkludering efter aggregering.

In [19]:
housing = pd.read_csv('housing.csv')
grouped = housing.groupby(['longitude', 'latitude'])
mean_cols = ['housing_median_age', 'median_income', 'median_house_value']
sum_cols = ['total_rooms', 'total_bedrooms', 'population', 'households']
num_cols = ['longitude', 'latitude', 'housing_median_age', 'median_income', 'median_house_value', 'total_rooms', 'total_bedrooms', 'population', 'households']
grouped['ocean_proximity'].nunique().value_counts()
ocean = grouped['ocean_proximity'].first()
aggregated = grouped[mean_cols].mean().join(grouped[sum_cols].sum()).join(ocean).reset_index()
aggregated

,longitude,latitude,housing_median_age,median_income,median_house_value,total_rooms,total_bedrooms,population,households,ocean_proximity
0,-124.35,40.54,52.0,3.0147,94600.0,1820.0,300.0,806.0,270.0,NEAR OCEAN
1,-124.30,41.80,19.0,1.9797,85800.0,2672.0,552.0,1298.0,478.0,NEAR OCEAN
2,-124.30,41.84,17.0,3.0313,103600.0,2677.0,531.0,1244.0,456.0,NEAR OCEAN
3,-124.27,40.69,36.0,2.5179,79000.0,2349.0,528.0,1194.0,465.0,NEAR OCEAN
4,-124.26,40.58,52.0,2.3571,111400.0,2217.0,394.0,907.0,369.0,NEAR OCEAN
...,...,...,...,...,...,...,...,...,...,...
12585,-114.56,33.69,17.0,1.6509,85700.0,720.0,174.0,333.0,117.0,INLAND
12586,-114.55,32.80,19.0,1.2750,56100.0,2570.0,820.0,1431.0,608.0,INLAND
12587,-114.49,33.97,17.0,1.6154,87500.0,2809.0,635.0,83.0,45.0,INLAND
12588,-114.47,34.40,19.0,1.8200,80100.0,7650.0,1901.0,1129.0,463.0,INLAND


kontroll av kategoriska variabler

In [20]:
grouped['ocean_proximity'].unique().value_counts()

ocean_proximity
[INLAND]        5264
[<1H OCEAN]     4719
[NEAR OCEAN]    1559
[NEAR BAY]      1043
[ISLAND]           5
Name: count, dtype: int64

one hot-encoding

In [21]:
aggregated = pd.get_dummies(aggregated, columns=['ocean_proximity'], drop_first=True)
X_cols = ['longitude', 'latitude', 'housing_median_age', 'median_income', 'total_rooms', 'total_bedrooms', 'population', 'households', 'ocean_proximity_INLAND', 'ocean_proximity_ISLAND', 'ocean_proximity_NEAR BAY', 'ocean_proximity_NEAR OCEAN']
aggregated

,longitude,latitude,housing_median_age,median_income,median_house_value,total_rooms,total_bedrooms,population,households,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-124.35,40.54,52.0,3.0147,94600.0,1820.0,300.0,806.0,270.0,False,False,False,True
1,-124.30,41.80,19.0,1.9797,85800.0,2672.0,552.0,1298.0,478.0,False,False,False,True
2,-124.30,41.84,17.0,3.0313,103600.0,2677.0,531.0,1244.0,456.0,False,False,False,True
3,-124.27,40.69,36.0,2.5179,79000.0,2349.0,528.0,1194.0,465.0,False,False,False,True
4,-124.26,40.58,52.0,2.3571,111400.0,2217.0,394.0,907.0,369.0,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12585,-114.56,33.69,17.0,1.6509,85700.0,720.0,174.0,333.0,117.0,True,False,False,False
12586,-114.55,32.80,19.0,1.2750,56100.0,2570.0,820.0,1431.0,608.0,True,False,False,False
12587,-114.49,33.97,17.0,1.6154,87500.0,2809.0,635.0,83.0,45.0,True,False,False,False
12588,-114.47,34.40,19.0,1.8200,80100.0,7650.0,1901.0,1129.0,463.0,True,False,False,False


In [22]:
correlation = aggregated[num_cols].corr()
correlation

,longitude,latitude,housing_median_age,median_income,median_house_value,total_rooms,total_bedrooms,population,households
longitude,1.000000,-0.910462,-0.097817,0.017519,-0.009701,0.112792,0.119243,0.148764,0.106515
latitude,-0.910462,1.000000,0.008430,-0.150570,-0.211825,-0.187946,-0.190898,-0.230928,-0.194128
housing_median_age,-0.097817,0.008430,1.000000,-0.113743,0.078236,-0.036467,0.041004,0.051028,0.064313
median_income,0.017519,-0.150570,-0.113743,1.000000,0.747927,0.119016,-0.062580,-0.062563,-0.046985
median_house_value,-0.009701,-0.211825,0.078236,0.747927,1.000000,0.200213,0.104872,0.038130,0.118080
total_rooms,0.112792,-0.187946,-0.036467,0.119016,0.200213,1.000000,0.926341,0.861363,0.924973
total_bedrooms,0.119243,-0.190898,0.041004,-0.062580,0.104872,0.926341,1.000000,0.899643,0.983233
population,0.148764,-0.230928,0.051028,-0.062563,0.038130,0.861363,0.899643,1.000000,0.923581
households,0.106515,-0.194128,0.064313,-0.046985,0.118080,0.924973,0.983233,0.923581,1.000000


bygger X och Y samt lägger till intercept-kolumnen $\beta_0$

In [31]:
X = aggregated[X_cols].to_numpy(dtype=float)
Y = aggregated['median_house_value'].to_numpy(dtype=float)
n = X.shape[0]
X = np.column_stack((np.ones(n), X))
d = X.shape[1] - 1
aggregated[X_cols]


,longitude,latitude,housing_median_age,median_income,total_rooms,total_bedrooms,population,households,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-124.35,40.54,52.0,3.0147,1820.0,300.0,806.0,270.0,False,False,False,True
1,-124.30,41.80,19.0,1.9797,2672.0,552.0,1298.0,478.0,False,False,False,True
2,-124.30,41.84,17.0,3.0313,2677.0,531.0,1244.0,456.0,False,False,False,True
3,-124.27,40.69,36.0,2.5179,2349.0,528.0,1194.0,465.0,False,False,False,True
4,-124.26,40.58,52.0,2.3571,2217.0,394.0,907.0,369.0,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
12585,-114.56,33.69,17.0,1.6509,720.0,174.0,333.0,117.0,True,False,False,False
12586,-114.55,32.80,19.0,1.2750,2570.0,820.0,1431.0,608.0,True,False,False,False
12587,-114.49,33.97,17.0,1.6154,2809.0,635.0,83.0,45.0,True,False,False,False
12588,-114.47,34.40,19.0,1.8200,7650.0,1901.0,1129.0,463.0,True,False,False,False


kör OLS-formeln

In [24]:
XtX = X.T @ X
XtX_invers = np.linalg.inv(XtX)
XtY = X.T @ Y

b_estimate = XtX_invers @ XtY


In [25]:
y_estimate = X @ b_estimate
residuals = Y - y_estimate
SSE = np.sum(residuals**2)


beräkning av $\hat{\sigma}^2$ och $RMSE$

In [26]:
variance = SSE / n - d - 1
STD = np.sqrt(variance)
RMSE = np.sqrt(SSE / n)
RMSE

np.float64(62532.76084603433)

Se över kodrutan nedan då det inte verkar bli rätt med talen där

In [43]:
C = XtX_invers * variance
t_stats = b_estimate / np.sqrt(np.diag(C))
from scipy.stats import t
p_values = 2 * t.sf(np.abs(t_stats), df=n-d-1)
len(C)

13